In [2]:
import pandas as pd
from modeller import *
import matplotlib.pyplot as plt

In [3]:
zott = pd.read_stata("zott22.dta")
zott.head()

,date,zeit,year,week,abs,noproms,proms,babs,zabs,preis,...,breit1,breit2,grptv,youtube,facebook,instagram,tiktok,programmatik,kamp,covid
0,2019-01-06,2019-01-06,2019,1,171.92,98.20,73.72,137.94,33.98,4.48,...,1.34,1.19,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
1,2019-01-13,2019-01-13,2019,2,242.79,139.21,103.57,160.01,82.78,4.25,...,1.32,1.16,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
2,2019-01-20,2019-01-20,2019,3,227.01,144.06,82.94,163.64,63.36,4.47,...,1.30,1.18,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
3,2019-01-27,2019-01-27,2019,4,233.12,163.80,69.31,184.39,48.73,4.64,...,1.30,1.18,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0
4,2019-02-03,2019-02-03,2019,5,318.48,144.99,173.49,186.59,131.89,4.21,...,1.29,1.18,0.0,0.0,0.0,0.0,0.0,0.0,0,0.0


## Ab hier Shaper

In [5]:
# Definition der Variablen
sales = "abs"
media = ["grptv", "youtube", "facebook", "programmatik", "instagram", "tiktok"]
media_type = ["tv", "olv", "social", "banner", "social", "olv"]


In [7]:
# Optimale AdStocks und Gamma
shape_params = shap_search(df=zott, target=sales, media_features=media, mtype=media_type, log=False)
shape_params

,AdStock,Gamma
grptv,0.900000,0.300000
youtube,0.777778,0.300000
facebook,0.300000,0.300000
programmatik,0.600000,0.300000
instagram,0.350000,0.633333
tiktok,0.666667,0.300000


In [8]:
# Speichern der Shape Variablen im Datensatz
for v in media:
    zott[f"shape_{v}"] = norm_transform(zott[v], adstock=shape_params.loc[v, "AdStock"], hill=shape_params.loc[v, "Gamma"])

## Ab hier dann Modeller


In [11]:
# Hier Fake Zahlen, im Echten Modellung naürlich die priors aus robyn nehmen
robyn = [0.55, 0.79, 0.79, 0.76, 0.93, 0.20]
shape_var = [c for c in zott.columns if c.startswith("shape_")]

In [12]:
Cont, Eff = modell_optimizer(df=zott, target="abs", impression_var=media, shape_var=shape_var, robyn_value=robyn, runs=1000)

In [14]:
Cont.sum().values / zott[media].sum().values

array([0.495     , 0.869     , 0.83388889, 0.836     , 0.85766667,
       0.19333333])

In [16]:
Eff

array([0.495     , 0.869     , 0.83388889, 0.836     , 0.85766667,
       0.19333333])

In [17]:
robyn

[0.55, 0.79, 0.79, 0.76, 0.93, 0.2]